In [1]:
import pandas as pd
transaction_df = pd.read_excel('Shopping-cart-dataset.xlsx')
grouped = transaction_df.groupby('Invoice')['Description'].apply(list)
new_df = pd.DataFrame({'Items': grouped})
transaction_df['Description'].fillna('Unknown', inplace=True)
grouped = transaction_df.groupby('Invoice')['Description'].apply(list)
new_df = pd.DataFrame({'Items': grouped})
new_df.reset_index(inplace=True)
new_df.rename(columns={'Invoice': 'TID'}, inplace=True)
print(new_df)
new_df = new_df.drop('TID', axis=1)
new_df
new_df.index.rename('TID', inplace=True)
new_df
new_df['Items'] = new_df['Items'].apply(lambda x: ','.join(map(str, x)))
new_df

new_df.index.rename('TID', inplace=True)
new_df.rename(columns={'Items' : 'item_list'}, inplace=True)
trans_df = new_df.item_list.str.split(',')
trans_df

           TID                                              Items
0     536365.0  [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
1     536366.0  [HAND WARMER UNION JACK, HAND WARMER RED POLKA...
2     536367.0  [ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO...
3     536368.0  [JAM MAKING SET WITH JARS, RED COAT RACK PARIS...
4     536369.0                         [BATH BUILDING BLOCK WORD]
...        ...                                                ...
2343   C540417  [CHILLI LIGHTS, TEA TIME PARTY BUNTING, PICNIC...
2344   C540470                      [WHITE TEA,COFFEE,SUGAR JARS]
2345   C540503  [CAKE STAND VICTORIAN FILIGREE LARGE, 3 HEARTS...
2346   C540527       [DOCTOR'S BAG SOFT TOY, GARDEN PATH JOURNAL]
2347   C540535  [PINK LARGE JEWELED PHOTOFRAME, WOOD 2 DRAWER ...

[2348 rows x 2 columns]


TID
0       [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
1       [HAND WARMER UNION JACK, HAND WARMER RED POLKA...
2       [ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO...
3       [JAM MAKING SET WITH JARS, RED COAT RACK PARIS...
4                              [BATH BUILDING BLOCK WORD]
                              ...                        
2343    [CHILLI LIGHTS, TEA TIME PARTY BUNTING, PICNIC...
2344                      [WHITE TEA, COFFEE, SUGAR JARS]
2345    [CAKE STAND VICTORIAN FILIGREE LARGE, 3 HEARTS...
2346         [DOCTOR'S BAG SOFT TOY, GARDEN PATH JOURNAL]
2347    [PINK LARGE JEWELED PHOTOFRAME, WOOD 2 DRAWER ...
Name: item_list, Length: 2348, dtype: object

In [2]:
from collections import Counter
def count_item(trans_items):
    flat_items = [item for trans in trans_items for item in trans]
    item_counts = Counter(flat_items)
    data = pd.DataFrame({'item_sets': list(item_counts.keys()), 'supp_count': list(item_counts.values())})
    data = data.sort_values('item_sets')
#     print('each item count ', data)
    return data

trans_df
df = pd.DataFrame(trans_df)
df
valid_items = ['WHITE HANGING HEART T-LIGHT HOLDER','WHITE METAL LANTERN','CREAM CUPID HEARTS COAT HANGER','KNITTED UNION FLAG HOT WATER BOTTLE','DOORMAT NEW ENGLAND']
mask = df['item_list'].apply(lambda x: any(item in x for item in valid_items))
count = mask.sum()
print(count)


mask = df['item_list'].apply(lambda x: any(item in x for item in valid_items))
filtered_df = df[mask]
# Pandas Series of filtered rows
filtered_series = filtered_df['item_list']
print(filtered_series)


419
TID
0       [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
2       [ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHO...
8       [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
10      [WHITE HANGING HEART T-LIGHT HOLDER, WHITE MET...
23      [CHRISTMAS LIGHTS 10 REINDEER, JAM MAKING SET ...
                              ...                        
2247    [WHITE HANGING HEART T-LIGHT HOLDER, CHARLIE +...
2277                 [WHITE HANGING HEART T-LIGHT HOLDER]
2311    [HANGING JAM JAR T-LIGHT HOLDER, WHITE HANGING...
2327    [MULTI COLOUR SILVER T-LIGHT HOLDER, WHITE MET...
2342                 [WHITE HANGING HEART T-LIGHT HOLDER]
Name: item_list, Length: 419, dtype: object


In [3]:
type(trans_df)
mmsdf = count_item(trans_df)
mmsdf.supp_count=mmsdf.supp_count/count
mmsdf

valid_items = ['WHITE HANGING HEART T-LIGHT HOLDER','WHITE METAL LANTERN','CREAM CUPID HEARTS COAT HANGER','KNITTED UNION FLAG HOT WATER BOTTLE','DOORMAT NEW ENGLAND']
mask = mmsdf['item_sets'].apply(lambda x: any(item in x for item in valid_items))
mmsdf = mmsdf[mask]
mmsdf

,item_sets,supp_count
2,CREAM CUPID HEARTS COAT HANGER,0.128878
20,DOORMAT NEW ENGLAND,0.121718
3,KNITTED UNION FLAG HOT WATER BOTTLE,0.341289
0,WHITE HANGING HEART T-LIGHT HOLDER,0.692124
1,WHITE METAL LANTERN,0.119332


In [4]:
def calculate_conf(value1, value2):
    return round(int(value1)/int(value2) * 100, 2)

def all_rules(freq_item_sets):
    count_data = count_item(trans_df)
    confidences = {}
    for row in freq_item_sets.item_sets:
        for i in range(len(row)):
            for j in range(len(row)):
                if i != j:
                    itemset_A = row[i]
                    itemset_B = row[j]
                    supp_count_A = count_data[count_data.item_sets == itemset_A].supp_count.item()
                    supp_count_AB = freq_item_sets[freq_item_sets.item_sets == row].supp_count.item()
                    conf = calculate_conf(supp_count_AB, supp_count_A)
                    confidences[(itemset_A, itemset_B)] = conf

    conf_df = pd.DataFrame(confidences.items(), columns=['item_set', 'confidence'])
    conf_df['item_set'] = conf_df['item_set'].apply(lambda x: tuple(sorted(x)))
    conf_df = conf_df.groupby('item_set').agg({'confidence': 'max'}).reset_index()
    return conf_df


def count_itemset(trans_data, itemsets):
    count_item = {}
    for item_set in itemsets:
        count_item[item_set] = sum([1 for transaction in trans_data if set(item_set).issubset(set(transaction))])
    data = pd.DataFrame()
    data['item_sets'] = count_item.keys()
    data['supp_count'] = count_item.values()
    return data


   
def joinmms(element, L):
    itemsets = []
    i = 1
    k=L.index(element)
    ss=len(L)-k
    while k!=len(L):
        if element!=L[k]:
            tuples = (element, L[k])
            itemsets.append(tuples)
        k=k+1
    return itemsets



# def join(list_of_items):
#     itemsets = []
#     for i, item1 in enumerate(list_of_items):
#         for j in range(i+1, len(list_of_items)):
#             item2 = list_of_items[j]
#             if type(item1) is str:
#                 tuples = (item1, item2)
#                 itemsets.append(tuples)
#             else:
#                 if item1[:-1] == item2[:-1]:
#                     tuples = item1 + (item2[-1],)
#                     itemsets.append(tuples)
#     if not itemsets:
#         return None
#     print('itemsets', itemsets)
#     return itemsets

# def joinmms(element, L, k):
#     itemsets = []
#     i = 1
#     index = L.index(element)
#     ss = len(L) - index
#     while index != len(L):
#         if element != L[index]:
#             remaining_list = L[index+1:]
#             candidate_stack = [(L[index],)]
#             # Generate candidates of size k-1 using a stack-based approach
#             while candidate_stack:
#                 candidate = candidate_stack.pop()
#                 if len(candidate) == k-1:
#                     itemsets.append((element,) + candidate)
#                 else:
#                     for j in range(len(remaining_list)-(k-len(candidate))+1):
#                         candidate_stack.append(candidate + (remaining_list[j],))
#         index += 1
#     return itemsets



def rulemis(l1):
    temp=1
    for i in l1:
        if temp>=M[i]:
            temp=M[i]
    return temp

def supdiff(l1):
    it1=l1[0]
    it2=l1[-1]
    x = mmsdf.loc[mmsdf['item_sets'] == it1]
    y = mmsdf.loc[mmsdf['item_sets'] == it2]
    temp= abs(x.iloc[0]['supp_count'] - y.iloc[0]['supp_count'])
    return temp


def validity_check1(pair):
#     print(M)
    i=1
    flag=0
    while i<len(pair):
#         print([pair[0]])
        x = mmsdf.loc[mmsdf['item_sets'] == pair[i]]
        if  x.iloc[0]['supp_count']>= M[pair[0]]:
            flag=1
        else:
            flag=0
        i=i+1
    if flag==0:
        return -1
    if flag==1:
        return 1
  

In [5]:
infrequent_items_mis=[]
frequent_items_mis=[]
def Multiple_Minimum_Support(MISVALUES_DICT, C,mui, iterations=5):
    for k in range(1, iterations+1):
        if k == 1:
            first_value = list(M.items())[0][1]
            L = []
            temp = []
            for key in M:
                temp.append(key)
            i = 0
            for index, row in mmsdf.iterrows():
                x = mmsdf.loc[mmsdf['item_sets'] == temp[i]]
                if x.iloc[0]['supp_count'] >= first_value:
                    print("CANDIDATES OF SIZE ", k ,temp[i])
                    L.append(temp[i])
                else:
                    print("PRUNED OF SIZE  ", k ,temp[i])
                    infrequent_items_mis.append(temp[i])
                    
                    
                i = i+1
            Frequent = []
            for element in L:
                x = mmsdf.loc[mmsdf['item_sets'] == element]
                if x.iloc[0]['supp_count'] >= M[element]:
                    print("FREQUENT QUALIFIED CANDIDATES OF SIZE ", k ,element)
                    Frequent.append(element)
                    frequent_items_mis.append(element)
                else:
                    
                    print("INFREQUENT CANDIDATES OF SIZE  ", k ,element)
                    infrequent_items_mis.append(element)
        else:
            candidates = []
            for element in L:
                
                x = mmsdf.loc[mmsdf['item_sets'] == element]
                if not x.empty and x.iloc[0]['supp_count'] >= M[element]:
                    its = joinmms(element, L,)
                    for i in its:
                        pair = list(i)
                        if validity_check1(pair) == 1 and supdiff(i) <= mui:
                            candidates.append(i)
                else:
                    continue
            L=list(candidates)
            varr=pd.DataFrame()
            temp = count_itemset(filtered_series, candidates)
            frequent_k = []
            for candidate in candidates:
                x = temp.loc[temp['item_sets'] == candidate]
                if x.iloc[0]['supp_count'] >= rulemis(list(x.iloc[0]['item_sets'])):
                    print("QUALIFIED FREQUENT OF SIZE ", k, list(x.iloc[0]['item_sets'])  )
                    varr = pd.concat([varr, x])
#                     print(varr)
                    frequent_k.append(list(x.iloc[0]['item_sets']))
                    frequent_items_mis.append(list(x.iloc[0]['item_sets']))
                else:
                    print("PRUNED ITEMSET OF SIZE ", k, list(x.iloc[0]['item_sets'])  )
                    infrequent_items_mis.append(list(x.iloc[0]['item_sets']))
              
            if  len(varr)>0:
                
                s = all_rules(varr)
                strong_rules = s[s.confidence >= C]
                weak_rules = s[s.confidence < C]
                print("=====================================STRONG- RULES LEVEL ",k,"==================================")
                print("strong_rules",strong_rules)
                print("=====================================WEAK- RULES LEVEL ",k,"==================================")
                print("weak_rules",weak_rules)
#             return weak_rules, strong_rules
            
        if k == iterations :
#             return
#             s = all_rules(temp)
#             strong_rules = s[s.confidence >= C]
#             weak_rules = s[s.confidence < C]
            return weak_rules, strong_rules


In [6]:
import time

start_time = time.time()

MISVALUES={'WHITE HANGING HEART T-LIGHT HOLDER':0.30,'WHITE METAL LANTERN':0.10,'CREAM CUPID HEARTS COAT HANGER':0.15,'KNITTED UNION FLAG HOT WATER BOTTLE':0.16,'DOORMAT NEW ENGLAND':0.20}
M={k: v for k, v in sorted(MISVALUES.items(), key=lambda item: item[1])}
weakrules,strongrules=Multiple_Minimum_Support(MISVALUES, 80.0,0.5)

end_time = time.time()
runtime = end_time - start_time
print("Runtime: {} seconds".format(runtime))


CANDIDATES OF SIZE  1 WHITE METAL LANTERN
CANDIDATES OF SIZE  1 CREAM CUPID HEARTS COAT HANGER
CANDIDATES OF SIZE  1 KNITTED UNION FLAG HOT WATER BOTTLE
CANDIDATES OF SIZE  1 DOORMAT NEW ENGLAND
CANDIDATES OF SIZE  1 WHITE HANGING HEART T-LIGHT HOLDER
FREQUENT QUALIFIED CANDIDATES OF SIZE  1 WHITE METAL LANTERN
INFREQUENT CANDIDATES OF SIZE   1 CREAM CUPID HEARTS COAT HANGER
FREQUENT QUALIFIED CANDIDATES OF SIZE  1 KNITTED UNION FLAG HOT WATER BOTTLE
INFREQUENT CANDIDATES OF SIZE   1 DOORMAT NEW ENGLAND
FREQUENT QUALIFIED CANDIDATES OF SIZE  1 WHITE HANGING HEART T-LIGHT HOLDER
QUALIFIED FREQUENT OF SIZE  2 ['WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER']
QUALIFIED FREQUENT OF SIZE  2 ['WHITE METAL LANTERN', 'KNITTED UNION FLAG HOT WATER BOTTLE']
QUALIFIED FREQUENT OF SIZE  2 ['WHITE METAL LANTERN', 'DOORMAT NEW ENGLAND']
QUALIFIED FREQUENT OF SIZE  2 ['KNITTED UNION FLAG HOT WATER BOTTLE', 'WHITE HANGING HEART T-LIGHT HOLDER']
=====================================STRONG- RULE

The item “CREAM CUPID HEARTS COAT HANGER”, was infrequent at level 1 but the itemset  
['WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER'] at level 2 came frequent itemset.


In [7]:
strongrules

,item_set,confidence


In [8]:
weakrules

,item_set,confidence
0,"(CREAM CUPID HEARTS COAT HANGER, WHITE METAL L...",36.00
1,"(DOORMAT NEW ENGLAND, WHITE METAL LANTERN)",2.00
2,"(KNITTED UNION FLAG HOT WATER BOTTLE, WHITE HA...",51.05
3,"(KNITTED UNION FLAG HOT WATER BOTTLE, WHITE ME...",50.00


In [9]:
frequent_items_mis

['WHITE METAL LANTERN',
 'KNITTED UNION FLAG HOT WATER BOTTLE',
 'WHITE HANGING HEART T-LIGHT HOLDER',
 ['WHITE METAL LANTERN', 'CREAM CUPID HEARTS COAT HANGER'],
 ['WHITE METAL LANTERN', 'KNITTED UNION FLAG HOT WATER BOTTLE'],
 ['WHITE METAL LANTERN', 'DOORMAT NEW ENGLAND'],
 ['KNITTED UNION FLAG HOT WATER BOTTLE', 'WHITE HANGING HEART T-LIGHT HOLDER']]

In [10]:
infrequent_items_mis

['CREAM CUPID HEARTS COAT HANGER', 'DOORMAT NEW ENGLAND']